# Practica 1 ABIA

<pre>
Este archivo contiene lo mismo que los otros archivos .py. Sirve para poder realizar la ejecución a trozos y así poder entenderlo mejor y 
hacer diferentes pruebas más fácilmente.
</pre>

<hr>
<hr>

### Imports

In [49]:
import random
from math import trunc
from tkinter import Y
from typing import List, Generator, Set
from search import Problem, hill_climbing

<hr>
<hr>

## abia_energy.py

### Constantes

<pre>Constantes comunes en todos los escenarios.</pre>

In [50]:
# Tipos de cliente
CLIENTE_XG = 0
CLIENTE_MG = 1
CLIENTE_G = 2

# Tipos de central
CENTRAL_A = 0
CENTRAL_B = 1
CENTRAL_C = 2

# Prioridades
GARANTIZADO = 0
NOGARANTIZADO = 1

# Producción por tipo de central
PROD = [[500.0, 250.0], [150.0, 100.0], [90.0, 10.0]]

# Consumo por tipo de cliente
CONSUMOS = [[15.0, 5.0], [3.0, 2.0], [2, 1]]

# Tabla de precios por tipo de cliente
# Primera dimensión: tipo de cliente
# Segunda dimensión, por orden: garantizado, no garantizado, indemnización
PRECIOS = [[40.0, 30.0, 5.0], [50.0, 40.0, 5.0], [60.0, 50.0, 5.0]]

# Tabla de costes
# Primera dimensión: tipo de central
# Segunda dimensión, por orden: coste por producción, coste diario, coste parada
COSTES = [[5, 2000, 1500], [8, 1000, 500], [15, 500, 150]]

# Tabla de pérdidas
# En cada posición, el primer valor es la distancia máxima del rango, el segundo valor es el tanto por uno de pérdida.
PERDIDA = [[10, 0], [25, 0.1], [50, 0.2], [75, 0.4], [1000, 0.6]]
TIPO = [0, 1, 2]
TIPOCL = [0, 1, 2]
TIPOCNT = [0, 1]

<hr>

### Clase Centrales

<pre>
Usamos esta clase para obtener una lista con las centrales generadas. 


Valores de entrada:
<ul>
<li>centrales_por_tipo: Una lista de 3 posiciones, con la cantidad de centrales de cada tipo que deseamos.</li>
<li>seed:               La semilla con la cual generaremos estas centrales, para así poder replicar este mismo experimento.</li></ul></pre>

In [51]:
class Centrales(list):
    
    def __init__(self, centrales_por_tipo: List[int], seed: int):
        if len(centrales_por_tipo) != 3:
            raise Exception("Vector Centrales de tamaño incorrecto")
        v_centrales = []
        rand = random.Random(seed)
        for i in range(3):
            for j in range(centrales_por_tipo[i]):
                p = (rand.random() * PROD[i][0]) + PROD[i][1]
                c = Central(TIPO[i], trunc(p), rand.randint(
                    0, 100), rand.randint(0, 100))
                v_centrales.append(c)
        super().__init__(v_centrales)

In [52]:
Centrales([2, 2, 2], 43)

[[Tipo=0|Produccion=269|cx=89|cy=97],
 [Tipo=0|Produccion=321|cx=59|cy=47],
 [Tipo=1|Produccion=200|cx=12|cy=58],
 [Tipo=1|Produccion=189|cx=77|cy=2],
 [Tipo=2|Produccion=56|cx=73|cy=47],
 [Tipo=2|Produccion=66|cx=70|cy=96]]

### Clase Central

<pre>
Usamos esta clase para generar cierto tipo de central.


Valores de entrada:
<ul>
<li>tipo:           El tipo de la central, un número del 0 al 2.</li>
<li>produccion:     Producción aleatoria dentro del rango del tipo de la central.</li>
<li>cx:             Coordenadas de la central en la posición X.</li>
<li>cy:             Coordenadas de la central en la posición Y.<li></ul></pre>

In [53]:
class Central(object):

    def __init__(self, tipo: int, produccion: float, cx: int, cy: int):
        self.Tipo = tipo  # Tipo de central
        self.Produccion = produccion  # Producción en MW
        self.CoordX = cx  # Coordenada x
        self.CoordY = cy  # Coordenada y

    def __repr__(self) -> str:
        return f"[Tipo={self.Tipo}|Produccion={self.Produccion}|cx={self.CoordX}|cy={self.CoordY}]"


In [54]:
Central(1, 256, 23, 33)

[Tipo=1|Produccion=256|cx=23|cy=33]

<hr>

### Clase Clientes

<pre>
Usamos esta clase para obtener una lista con los clientes generados. 


Valores de entrada:
<ul>
<li>ncl:    Número de clientes que deseemos.</li>
<li>propc:  Lista de 3 posiciones con la proporción de los 3 tipos de clientes.</li>
<li>propg:  Proporción de clientes garantizados.</li>
<li>seed:   La semilla con la cual generaremos estas centrales, para así poder replicar este mismo experimento.</li></ul></pre>

In [55]:
class Clientes(list):

    def __init__(self, ncl: int, propc: List[float], propg: float, seed: int):
        if len(propc) != 3:
            raise Exception(
                "Vector proporciones tipos clientes de tamaño incorrecto")
        if (propc[0] + propc[1] + propc[2]) != 1.0:
            raise Exception("Vector proporciones tipos clientes no suma 1")
        if 0.0 > propg > 1.0:
            raise Exception("Proporcion garantizado fuera de limites")

        v_clientes = []
        rand = random.Random(seed + 1)

        for i in range(ncl):
            dice = rand.random()
            if dice < propc[0]:
                rd = 0
            elif dice < (propc[0] + propc[1]):
                rd = 1
            else:
                rd = 2

            dice = rand.random()
            if dice < propg:
                rc = 0
            else:
                rc = 1

            c = (rand.random() * CONSUMOS[rd][0]) + CONSUMOS[rd][1]
            v_clientes.append(
                Cliente(TIPOCL[rd], trunc(c), TIPOCNT[rc], rand.randint(0, 100), rand.randint(0, 100)))
        super().__init__(v_clientes)

In [56]:
Clientes(10, [0.5, 0.25, 0.25], 0.5, 43)

[[Tipo=0|Consumo=17|Contrato=1|CoordX=22|CoordY=48],
 [Tipo=0|Consumo=6|Contrato=0|CoordX=1|CoordY=12],
 [Tipo=0|Consumo=19|Contrato=1|CoordX=88|CoordY=81],
 [Tipo=0|Consumo=15|Contrato=0|CoordX=41|CoordY=45],
 [Tipo=0|Consumo=14|Contrato=1|CoordX=14|CoordY=65],
 [Tipo=0|Consumo=19|Contrato=0|CoordX=13|CoordY=80],
 [Tipo=2|Consumo=1|Contrato=0|CoordX=12|CoordY=61],
 [Tipo=0|Consumo=6|Contrato=0|CoordX=95|CoordY=31],
 [Tipo=2|Consumo=2|Contrato=0|CoordX=87|CoordY=55],
 [Tipo=1|Consumo=4|Contrato=1|CoordX=83|CoordY=42]]

### Clase Cliente

<pre>
Usamos esta clase para guardar cierto tipo de cliente.


Valores de entrada:
<ul>
<li>t:      Tipo de cliente, número del 0 al 2.</li>
<li>cons:   Consumo del tipo de cliente, también aleatorio dentro del rango de dicho tipo.</li>
<li>cont:   Tipo de contrato del cliente.</li>
<li>cx:     Coordenadas de la central en la posición X.</li>
<li>cy:     Coordenadas de la central en la posición Y.</li></ul></pre>

In [57]:
class Cliente(object):
    
    def __init__(self, t: int, cons: float, cont: int, cx: int, cy: int):
        self.Tipo = t  # Tipo del cliente
        self.Consumo = cons  # Consumo demandado
        self.Contrato = cont  # Tipo de contrato
        self.CoordX = cx  # Coordenada x
        self.CoordY = cy  # Coordenada y

    def __repr__(self) -> str:
        return f"[Tipo={self.Tipo}|Consumo={self.Consumo}|Contrato={self.Contrato}|CoordX={self.CoordX}|CoordY={self.CoordY}]"


In [58]:
Cliente(2, 250, 0, 22, 44)

[Tipo=2|Consumo=250|Contrato=0|CoordX=22|CoordY=44]

<hr>

### Clase VEnergia

<pre>
Clase estática que permite acceder a los valores de los costes de las centrales y las tarifas y penalizaciones de los clientes.
</pre>

In [59]:
class VEnergia(object):

    @staticmethod
    def tarifa_cliente_garantizada(tipo: int) -> float:
        
        """ Retorna la tarifa de un cliente con servicio garantizado. """

        if tipo < 0 or tipo > 2:
            raise Exception("Tipo fuera de rango")
        else:
            return PRECIOS[tipo][0]

    @staticmethod
    def tarifa_cliente_no_garantizada(tipo: int) -> float:
        
        """ Retorna la tarifa de un cliente con servicio no garantizado. """
        
        if tipo < 0 or tipo > 2:
            raise Exception("Tipo fuera de rango")
        else:
            return PRECIOS[tipo][1]

    @staticmethod
    def tarifa_cliente_penalizacion(tipo: int) -> float:
        
        """ Retorna la penalización por servir a un cliente con servicio no garantizado. """
        
        if tipo < 0 or tipo > 2:
            raise Exception("Tipo fuera de rango")
        else:
            return PRECIOS[tipo][2]

    @staticmethod
    def costs_production_mw(tipo: int) -> float:
        
        """ Retorna el coste de producción por MW para una central de un tipo. """
        
        if tipo < 0 or tipo > 2:
            raise Exception("Tipo fuera de rango")
        else:
            return COSTES[tipo][0]

    @staticmethod
    def daily_cost(tipo: int) -> float:
        
        """ Retorna el coste de una central en marcha segun su tipo. """
        
        if tipo < 0 or tipo > 2:
            raise Exception("Tipo fuera de rango")
        else:
            return COSTES[tipo][1]

    @staticmethod
    def stop_cost(tipo: int) -> float:
        
        """ Retorna el coste de una central en parada segun su tipo. """
        
        if tipo < 0 or tipo > 2:
            raise Exception("Tipo fuera de rango")
        else:
            return COSTES[tipo][2]

    @staticmethod
    def loss(distancia: float) -> float:
        
        """ Retorna la perdida (en tanto por uno) segun la distancia entre central y cliente. """
        
        i = 0
        while distancia > PERDIDA[i][0]:
            i = i + 1
        return PERDIDA[i][1]

<hr>
<hr>

## abia_energy_problem.py

### Parámetros del problema

<pre>
En este caso nuestros parametros son la lista de CLIENTES y la lista de CENTRALES.
</pre>

In [60]:
class ProblemParameters(object):
    def __init__(self, clients_vector: Clientes, power_plants_vector: Centrales) -> None:
        self.clients_vector = clients_vector
        self.power_plants_vector = power_plants_vector

    def __repr__(self):
        return f"clients_vector={self.clients_vector}\n\npower_plants_vector={self.power_plants_vector})"


In [61]:
clientes = Clientes(ncl=10, propc=[0.5, 0.25, 0.25], propg=0.5, seed=43)
centrales = Centrales(centrales_por_tipo=[1, 1, 1], seed=43)
problem_parameters = ProblemParameters(clients_vector=clientes, power_plants_vector=centrales)
problem_parameters

clients_vector=[[Tipo=0|Consumo=17|Contrato=1|CoordX=22|CoordY=48], [Tipo=0|Consumo=6|Contrato=0|CoordX=1|CoordY=12], [Tipo=0|Consumo=19|Contrato=1|CoordX=88|CoordY=81], [Tipo=0|Consumo=15|Contrato=0|CoordX=41|CoordY=45], [Tipo=0|Consumo=14|Contrato=1|CoordX=14|CoordY=65], [Tipo=0|Consumo=19|Contrato=0|CoordX=13|CoordY=80], [Tipo=2|Consumo=1|Contrato=0|CoordX=12|CoordY=61], [Tipo=0|Consumo=6|Contrato=0|CoordX=95|CoordY=31], [Tipo=2|Consumo=2|Contrato=0|CoordX=87|CoordY=55], [Tipo=1|Consumo=4|Contrato=1|CoordX=83|CoordY=42]]

power_plants_vector=[[Tipo=0|Produccion=269|cx=89|cy=97], [Tipo=1|Produccion=121|cx=59|cy=47], [Tipo=2|Produccion=70|cx=12|cy=58]])

<hr>

### Operadores

In [62]:
class Operator(object):
    pass

<pre>
Tenemos 3 operadores:

<b><i>- MoveClient</i></b>

    Nos sirve para mover a un cliente a una central.
</pre>

In [63]:
class MoveClient(Operator):
    def __init__(self, id_client, id_destination_PwP):
        self.id_client = id_client
        self.id_destination_PwP = id_destination_PwP

    def __repr__(self) -> str:
        return f"Client {self.id_client} has been moved to power plant {self.id_destination_PwP}"

In [64]:
MoveClient(0, 0)  # El cliente de la posición 0 del vector se ha movido a la central de posición 0 del vector.

Client 0 has been moved to power plant 0

<pre>
<b><i>- SwapClient</i></b>

    Nos sirve para intercambiar dos clientes entre dos centrales.
</pre>

In [65]:
class SwapClients(Operator):
    def __init__(self, id_client1, id_client2):
        self.id_client1 = id_client1
        self.id_client2 = id_client2

    def __repr__(self) -> str:
        return f"Swap between Client {self.id_client1} and Client {self.id_client2}"

In [66]:
SwapClients(0, 1)  # El cliente de la posición 0 del vector se intercambia con el cliente de la posición 1.

Swap between Client 0 and Client 1

<pre>
<b><i>- RemoveNGClient</i></b>

    Nos sirve para eliminar a un cliente NO GARANTIZADO de una central.
</pre>

In [67]:
class RemoveNGClient(Operator):
    def __init__(self, p_i: int, p_j: int):
        pass

    def __repr__(self) -> str:
        pass

<hr>

### Nuestros métodos

<pre>
Funciones propias que nos sirven para poder realizar el problema.
</pre>

In [68]:
def distance(obj1, obj2):
    x = obj1.CoordX - obj2.CoordX
    y = obj1.CoordY - obj2.CoordY
    return (x**2 + y ** 2)**(1/2)

def electry_supplied_to_client(client: Cliente, power_plant: Central) -> int:
    dist = distance(client, power_plant)
    return client.Consumo * (1 + VEnergia.loss(dist))

In [69]:
print(problem_parameters.clients_vector[0])
print(problem_parameters.power_plants_vector[0])
print(electry_supplied_to_client(problem_parameters.clients_vector[0], problem_parameters.power_plants_vector[0]))

[Tipo=0|Consumo=17|Contrato=1|CoordX=22|CoordY=48]
[Tipo=0|Produccion=269|cx=89|cy=97]
27.200000000000003


<hr>

### Representación del estado

<pre>
Nuestra representación del estado del problema, con las diversas funciones ......
</pre>

In [70]:
class StateRepresentation(object):
    def __init__(self, params: ProblemParameters, client_power_plant: List[int], remaining_energies: List[float], real_consumption: List[List[float]]):
        self.params = params
        self.client_power_plant = client_power_plant
        self.remaining_energies = remaining_energies
        self.real_consumption = real_consumption

    def copy(self):
        return StateRepresentation(self.params, self.client_power_plant.copy(), self.remaining_energies.copy(), self.real_consumption.copy())

    def __repr__(self) -> str:
        return f"client_power_plant: {self.client_power_plant}"

    def generate_actions(self) -> Generator[Operator, None, None]:
        for id_client1 in range(len(self.params.clients_vector)):
            for id_PwP in range(len(self.params.power_plants_vector)):
                if self.client_power_plant[id_client1] == id_PwP:
                    continue

                csm_client = self.real_consumption[id_PwP][id_client1]

                if csm_client < self.remaining_energies[id_PwP]:
                    yield MoveClient(id_client1, id_PwP)

            for id_client2 in range(len(self.params.clients_vector)):
                if id_client1 == id_client2:
                    continue

                id_PwP1 = self.client_power_plant[id_client1]
                id_PwP2 = self.client_power_plant[id_client2]

                if id_PwP1 == id_PwP2:
                    continue

                if id_PwP1 == -1 or id_PwP2 == -1:
                    continue

                csm_pwp1_cli1 = self.real_consumption[id_PwP1][id_client1]
                csm_pwp2_cli1 = self.real_consumption[id_PwP2][id_client1]

                csm_pwp1_cli2 = self.real_consumption[id_PwP1][id_client2]
                csm_pwp2_cli2 = self.real_consumption[id_PwP2][id_client2]

                remain1 = self.remaining_energies[id_PwP1]
                remain2 = self.remaining_energies[id_PwP2]

                if (csm_pwp1_cli2 - csm_pwp1_cli1 < remain1 and csm_pwp2_cli1 - csm_pwp2_cli2 < remain2):
                    yield SwapClients(id_client1, id_client2)

    def apply_actions(self, action: Operator):  # -> StateRepresentation:
        new_state = self.copy()

        if isinstance(action, MoveClient):
            id_client = action.id_client
            id_PwP1 = self.client_power_plant[id_client]
            id_PwP2 = action.id_destination_PwP
            new_state.client_power_plant[id_client] = id_PwP2 = action.id_destination_PwP

            if id_PwP1 != -1:
                new_state.remaining_energies[id_PwP1] += self.real_consumption[id_PwP1][id_client]

            new_state.remaining_energies[id_PwP2] -= self.real_consumption[id_PwP2][id_client]

        elif isinstance(action, SwapClients):
            id_client1 = action.id_client1
            id_client2 = action.id_client2

            id_PwP1 = self.client_power_plant[id_client1]
            id_PwP2 = self.client_power_plant[id_client2]

            new_state.client_power_plant[id_client1] = id_PwP2
            new_state.client_power_plant[id_client2] = id_PwP1

            new_state.remaining_energies[id_PwP1] += self.real_consumption[id_PwP1][id_client1] - self.real_consumption[id_PwP1][id_client2]
            new_state.remaining_energies[id_PwP2] += self.real_consumption[id_PwP2][id_client2] - self.real_consumption[id_PwP2][id_client1]

        return new_state

    def heuristic(self) -> float:
        gain = 0

        for id_central, central in enumerate(self.params.power_plants_vector):
            if central.Produccion == self.remaining_energies[id_central]:
                gain -= VEnergia.stop_cost(central.Tipo)
            else:
                gain -= VEnergia.costs_production_mw(
                    central.Tipo) * central.Produccion + VEnergia.daily_cost(central.Tipo)

        for id_client, client in enumerate(self.params.clients_vector):
            if self.client_power_plant[id_client] == -1:
                gain -= VEnergia.tarifa_cliente_penalizacion(
                    client.Tipo) * client.Consumo
                continue

            if client.Contrato == 0:
                gain += VEnergia.tarifa_cliente_garantizada(
                    client.Tipo) * client.Consumo
            else:
                gain += VEnergia.tarifa_cliente_no_garantizada(
                    client.Tipo) * client.Consumo

        return gain

<hr>

### Generación del estado inicial

<pre>
Diferentes funciones para crear diferentes casos de estado inicial.

<b><i>- generate_simple_initial_state</i></b>

    Generación de un estado inicial simple en el que metemos todos los clientes GARANTIZADOS posibles en orden en una central hasta que
    no quepan más por falta de producción, entonces metemos en la siguiente central. Pueden quedar centrales vacias. También nos guardamos 
    una lista con la energia no utilizada. 
</pre>

In [71]:
def generate_simple_initial_state(params: ProblemParameters):
    remaining_energies = list()
    real_consumption = list()
    client_power_plant = list()

    ids_power_plants = list()

    for id_PwP, power_plant in enumerate(params.power_plants_vector):
        ids_power_plants.append(id_PwP)
        remaining_energies.append(power_plant.Produccion)
        row = list()

        for client in params.clients_vector:
            consumtion = electry_supplied_to_client(client, power_plant)
            row.append(consumtion)
        real_consumption.append(row)

    id_PwP = ids_power_plants.pop()

    for id_client, client in enumerate(params.clients_vector):
        if client.Contrato == NOGARANTIZADO:
            client_power_plant.append(-1)
            continue

        consum = real_consumption[id_PwP][id_client]
        while True:
            if consum < remaining_energies[id_PwP]:
                client_power_plant.append(id_PwP)
                remaining_energies[id_PwP] -= consum
                break

            id_PwP = ids_power_plants.pop()

    return StateRepresentation(params, client_power_plant, remaining_energies, real_consumption)

In [72]:
clientes = Clientes(ncl=3, propc=[1, 0, 0], propg=1, seed=44)
centrales = Centrales(centrales_por_tipo=[1, 1, 1], seed=44)
parametros = ProblemParameters(clients_vector=clientes, power_plants_vector=centrales)
estado = generate_simple_initial_state(params=parametros)

print(estado)

client_power_plant: [2, 2, 1]


<pre>
<b><i>- generate_complex_initial_state</i></b>

    Generación de un estado inicial más complejo con ...
</pre>

In [73]:
def generate_complex_initial_state():
    pass

<hr>

### Resolver el problema creando una clase

<pre>
Con esta clase podremos llamar a la función hill_climbing para poder resolver el problema y obtener la solución.
</pre>

In [74]:
class EnergyProblem(Problem):
    def __init__(self, initial_state: StateRepresentation):
        super().__init__(initial_state)
        
    def actions(self, state: StateRepresentation) -> Generator[Operator,None,None]:
        return state.generate_actions()
    
    def result(self, state: StateRepresentation, action: Operator) -> StateRepresentation:
        return state.apply_actions(action)
    
    def value(self, state: StateRepresentation) -> float:
        return state.heuristic()
        
    def goal_test(self, state: StateRepresentation) -> bool:
        return False

<hr>
<hr>

### Ejecución del programa y resolución del problema

In [75]:
clientes = Clientes(ncl=100, propc=[0.4, 0.3, 0.3], propg=1, seed=44)
centrales = Centrales(centrales_por_tipo=[2, 1, 1], seed=44)
parametros = ProblemParameters(clients_vector=clientes, power_plants_vector=centrales)
estado = generate_simple_initial_state(params=parametros)

#print(estado)

#print("\n------------------------------------------------")
#print(list(estado.generate_actions()))

print("\n------------------------------------------------")
print(estado.heuristic())


------------------------------------------------
15341.0


<pre>
Ejecución del hill climbing para resolver el problema.
</pre>

In [76]:
print(hill_climbing(EnergyProblem(estado)))

client_power_plant: [3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
